In [1]:
%load_ext kedro.ipython
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)

import altair as alt
import vl_convert as vlc
alt.data_transformers.disable_max_rows()

from alphafold_impact.settings import SESSION_STORE_ARGS
PATHDIR = SESSION_STORE_ARGS['path']
SAVE_DIR = PATHDIR + '/data/08_reporting/reach/'
ECR_SAVE_DIR = PATHDIR + '/data/08_reporting/reach/ecr/'

# check folder, create if it doesn't exist
import os
for folder in [SAVE_DIR, ECR_SAVE_DIR]:
    if not os.path.exists(folder):
        os.makedirs(folder)

[06/23/25 10:01:28] INFO     Resolved project path as: /home/ampudia/projects/alphafold-impact.     ]8;id=776649;file:///home/ampudia/miniconda3/envs/.igl-alphafold/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=431889;file:///home/ampudia/miniconda3/envs/.igl-alphafold/lib/python3.11/site-packages/kedro/ipython/__init__.py#146\146]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[06/23/25 10:01:28] WARNING  Top-level credentials will not propagate into the           ]8;id=555922;file:///home/ampudia/miniconda3/envs/.igl-alphafold/lib/python3.11/site-packages/kedro_datasets/partitions/partitioned_dataset.py\partitioned_dataset.py]8;;\:]8;id=191786;file:///home/ampudia/miniconda3/envs/.igl-alphafold/lib/python3.11/site-packages/kedro_datasets/partitions/partitioned_dataset.py#215\215]8;;\
                             underlying dataset since credentials were explicitly                                  
                             defined in the underlying dataset config.                                             

[06/23/25 10:01:29] INFO     Kedro project alphafold-impact                                         ]8;id=589085;file:///home/ampudia/miniconda3/envs/.igl-alphafold/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=687672;file:///home/ampudia/miniconda3/envs/.igl-alphafold/lib/python3.11/site-packages/kedro/ipython/__init__.py#115\115]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=860504;file:///home/ampudia/miniconda3/envs/.igl-alphafold/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=732096;file:///home/ampudia/miniconda3/envs/.igl-alphafold/lib/python3.11/site-packages/kedro/ipython/__init__.py#116\116]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=914821;file:///home/ampudia/miniconda3/envs/.igl-alphafold/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=305119;file:///home/ampudia/miniconda3/envs/.igl-alphafold/lib/python3.11/site-packages/kedro/ipython/__init__.py#122\122]8;;\

In [2]:
# Reach

In [2]:
publications = catalog.load("publications.data.outputs")

[06/23/25 10:01:33] INFO     Loading data from publications.data.outputs (ParquetDataset)...    ]8;id=25985;file:///home/ampudia/miniconda3/envs/.igl-alphafold/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=357560;file:///home/ampudia/miniconda3/envs/.igl-alphafold/lib/python3.11/site-packages/kedro/io/data_catalog.py#482\482]8;;\

In [3]:
publications["authors"] = publications["authorships"].apply(
    lambda x: [y[0] for y in x] if x is not None else []
)

In [4]:
publications["publication_date"] = pd.to_datetime(publications["publication_date"])
publications["quarter"] = publications["publication_date"].dt.to_period("Q").astype(str)

In [6]:
authors = (
    publications[
        [
            "id",
            "source",
            "authors",
            "level",
            "quarter",
            "fwci",
            "primary_field"
        ]
    ]
    .explode("authors")
    .drop_duplicates(subset=["authors", "source"])
)


In [7]:
authors = (
    publications[
        [
            "id",
            "source",
            "authors",
            "level",
            "quarter",
            "fwci",
            "primary_field"
        ]
    ]
    .explode("authors")
    .drop_duplicates(subset=["authors", "source"])
)

unique_researchers_overtime = (
    authors.groupby(["quarter", "source"])["authors"]
    .nunique()
    .reset_index()
    .rename(columns={"authors": "researcher_count"})
)

unique_researchers_overtime["cumulative_researcher_count"] = (
    unique_researchers_overtime.groupby("source")["researcher_count"].cumsum()
)

unique_researchers_overtime = unique_researchers_overtime[
    (unique_researchers_overtime["quarter"] >= "2020Q1")
    & (unique_researchers_overtime["quarter"] <= "2024Q1")
]

chart = alt.Chart(unique_researchers_overtime).mark_line(interpolate='step-after').encode(
    x='quarter:N',
    y='cumulative_researcher_count:Q',
    color='source:N',
    tooltip=['quarter:N', 'cumulative_researcher_count:Q', 'source:N']
).properties(
    title='Number of Source-Unique Researchers Over Time by Source Group'
)

png_str = vlc.vegalite_to_png(vl_spec=chart.to_json(), scale=3)
with open("source_unique_researchers.png", "wb") as f:
    f.write(png_str)

In [8]:
# Assign priority to each source
source_priority = {"af": 1, "ct_ai": 2, "ct_noai": 3, "other": 4}
authors["source_priority"] = authors["source"].map(source_priority)

# Sort the DataFrame based on the priority
authors = authors.sort_values(by=["authors", "source_priority"])

# Drop duplicates based on authors to keep the highest priority source for each author
authors = authors.drop_duplicates(subset=["authors"])

# Group by quarter and count the number of unique researchers
unique_researchers_overtime = (
    authors.groupby(["quarter", "source"])["authors"]
    .nunique()
    .reset_index()
    .rename(columns={"authors": "researcher_count"})
)

# Calculate the cumulative count of unique researchers over time for each source
unique_researchers_overtime["cumulative_researcher_count"] = (
    unique_researchers_overtime.groupby("source")["researcher_count"].cumsum()
)

# Filter the data for the desired time range
unique_researchers_overtime = unique_researchers_overtime[
    (unique_researchers_overtime["quarter"] >= "2020Q1")
    & (unique_researchers_overtime["quarter"] <= "2024Q1")
]

chart = alt.Chart(unique_researchers_overtime).mark_line(interpolate='step-after').encode(
    x='quarter:N',
    y='cumulative_researcher_count:Q',
    color='source:N',
    tooltip=['quarter:N', 'cumulative_researcher_count:Q', 'source:N']
).properties(
    title='Number of Unique Researchers Over Time by Source Group'
)

png_str = vlc.vegalite_to_png(vl_spec=chart.to_json(), scale=3)
with open("unique_researchers.png", "wb") as f:
    f.write(png_str)

In [9]:
authors = authors[authors["primary_field"].isin(["Biochemistry, Genetics and Molecular Biology", "Medicine", "Chemistry", "Immunology and Microbiology"])]

unique_researchers_overtime = (
    authors.groupby(["quarter", "source", "primary_field"])["authors"]
    .nunique()
    .reset_index()
    .rename(columns={"authors": "researcher_count"})
)

# Calculate the cumulative count of unique researchers over time for each source and primary_field
unique_researchers_overtime["cumulative_researcher_count"] = (
    unique_researchers_overtime.groupby(["source", "primary_field"])[
        "researcher_count"
    ].cumsum()
)

# Filter the data for the desired time range
unique_researchers_overtime = unique_researchers_overtime[
    (unique_researchers_overtime["quarter"] >= "2020Q1")
    & (unique_researchers_overtime["quarter"] <= "2024Q1")
]

# Create the Altair step chart with faceting by primary_field
step_line = (
    alt.Chart(unique_researchers_overtime)
    .mark_line(interpolate="step-after")
    .encode(
        alt.X("quarter:N", title="Quarter"),
        alt.Y("cumulative_researcher_count:Q", title=None),
        alt.Color("source:N", title="Source"),
    )
)

chart = (
    step_line.properties(
        title="Number of Source-Unique Researchers Over Time by Field",
        width=300,
        height=120,
    )
    .facet(facet="primary_field:N", columns=2, spacing=25)
    .resolve_scale(x="shared", y="independent")
    .configure_title(fontSize=16)
    .configure_axis(labelFontSize=12, titleFontSize=14)
    .configure_header(titleFontSize=14, labelFontSize=12)
)

png_str = vlc.vegalite_to_png(vl_spec=chart.to_json(), scale=3)
with open("source_unique_researchers_field.png", "wb") as f:
    f.write(png_str)

In [7]:
# create mesh_C_ind for mesh_C that is 1 if mesh_C is > 0, 0 otherwise
publications["mesh_C_ind"] = publications["mesh_C"].apply(lambda x: 1 if x > 0 else 0)

In [8]:
# create variable depth that is "adjacent" if level is 0, and "downstream" if level is 1 or 2
publications["depth"] = publications["level"].apply(lambda x: "adjacent" if x == "0" else "downstream" if x in [1, 2] else "distant")

In [9]:
publications.groupby(["source", "depth"])["mesh_C_ind"].sum()


source  depth   
af      adjacent     3097
        distant     76320
ct_ai   adjacent      644
        distant     19661
ct_pp   adjacent     1865
        distant     53541
ct_sb   adjacent     1383
        distant     49112
other   adjacent     2058
        distant     92698
Name: mesh_C_ind, dtype: int64

In [10]:
publications.groupby(["source", "depth"])["mesh_C_ind"].mean()


source  depth   
af      adjacent    0.094392
        distant     0.148382
ct_ai   adjacent    0.116582
        distant     0.122388
ct_pp   adjacent    0.286219
        distant     0.242117
ct_sb   adjacent    0.221919
        distant     0.244117
other   adjacent    0.069009
        distant     0.185471
Name: mesh_C_ind, dtype: float64

In [11]:
strong_publications = publications[publications["chain_label"].isin(["strong", "partial_strong"])]
strong_publications.groupby(["source", "depth"])["mesh_C_ind"].sum()


source  depth   
af      adjacent      685
        distant     15673
ct_ai   adjacent        0
        distant      1508
ct_pp   adjacent       13
        distant      3330
ct_sb   distant      2094
other   adjacent       39
        distant      6591
Name: mesh_C_ind, dtype: int64

In [12]:
strong_publications.groupby(["source", "depth"])["mesh_C_ind"].mean()


source  depth   
af      adjacent    0.131883
        distant     0.195275
ct_ai   adjacent    0.000000
        distant     0.155432
ct_pp   adjacent    0.541667
        distant     0.281274
ct_sb   distant     0.264528
other   adjacent    0.096296
        distant     0.211365
Name: mesh_C_ind, dtype: float64

In [13]:
publications.ca_count


0          4.0
1          0.0
2          1.0
3          0.0
4          0.0
          ... 
1678008    0.0
1678009    0.0
1678010    0.0
1678011    0.0
1678012    0.0
Name: ca_count, Length: 1678013, dtype: float64

#### Clinical citations

In [14]:
publications.groupby(["source", "depth"])["ca_count"].sum()



source  depth   
af      adjacent     22634.0
        distant     313889.0
ct_ai   adjacent      3692.0
        distant      77918.0
ct_pp   adjacent      4702.0
        distant     144460.0
ct_sb   adjacent      4726.0
        distant     135673.0
other   adjacent     16928.0
        distant     321294.0
Name: ca_count, dtype: float64

In [25]:
publications.groupby(["source", "depth"])["ca_count"].mean()


source   depth   
af       adjacent    0.004747
         distant     0.018750
ct_ai    adjacent    0.006520
         distant     0.005714
ct_noai  adjacent    0.034029
         distant     0.179203
other    adjacent    0.001401
         distant     0.018398
Name: ca_count, dtype: float64

In [26]:
strong_publications.groupby(["source", "depth"])["ca_count"].sum()


source   depth   
af       adjacent      32.0
         distant     1776.0
ct_ai    adjacent       4.0
         distant       20.0
ct_noai  adjacent      18.0
         distant     1615.0
other    distant      273.0
Name: ca_count, dtype: float64

In [28]:
strong_publications.groupby(["source", "depth"])["ca_count"].mean()


source   depth   
af       adjacent    0.009849
         distant     0.048554
ct_ai    adjacent    0.006107
         distant     0.004879
ct_noai  adjacent    0.031469
         distant     0.176503
other    distant     0.014097
Name: ca_count, dtype: float64

#### Patent counts

In [29]:
publications.groupby(["source", "depth"])["patent_count"].sum()



source   depth   
af       adjacent      79.0
         distant      461.0
ct_ai    adjacent      76.0
         distant      225.0
ct_noai  adjacent     194.0
         distant     2318.0
other    adjacent     705.0
         distant     2389.0
Name: patent_count, dtype: float64

In [30]:
publications.groupby(["source", "depth"])["patent_count"].mean()


source   depth   
af       adjacent    0.006579
         distant     0.002595
ct_ai    adjacent    0.026081
         distant     0.006429
ct_noai  adjacent    0.055476
         distant     0.029069
other    adjacent    0.070564
         distant     0.008174
Name: patent_count, dtype: float64

In [31]:
strong_publications.groupby(["source", "depth"])["patent_count"].sum()


source   depth   
af       adjacent     25.0
         distant     148.0
ct_ai    adjacent      7.0
         distant      27.0
ct_noai  adjacent     28.0
         distant     438.0
other    distant     167.0
Name: patent_count, dtype: float64

In [32]:
strong_publications.groupby(["source", "depth"])["patent_count"].mean()


source   depth   
af       adjacent    0.007695
         distant     0.004046
ct_ai    adjacent    0.010687
         distant     0.006587
ct_noai  adjacent    0.048951
         distant     0.047869
other    distant     0.008623
Name: patent_count, dtype: float64

#### Patent citations

In [34]:
publications.groupby(["source", "depth"])["patent_citation"].sum()



source   depth   
af       adjacent       9.0
         distant       88.0
ct_ai    adjacent      27.0
         distant       48.0
ct_noai  adjacent     148.0
         distant     2191.0
other    adjacent     817.0
         distant     1045.0
Name: patent_citation, dtype: float64

In [35]:
publications.groupby(["source", "depth"])["patent_citation"].mean()


source   depth   
af       adjacent    0.132353
         distant     0.251429
ct_ai    adjacent    0.551020
         distant     0.279070
ct_noai  adjacent    1.525773
         distant     1.740270
other    adjacent    3.071429
         distant     0.585434
Name: patent_citation, dtype: float64

In [36]:
strong_publications.groupby(["source", "depth"])["patent_citation"].sum()


source   depth   
af       adjacent      1.0
         distant      25.0
ct_ai    adjacent      0.0
         distant      10.0
ct_noai  adjacent      8.0
         distant     583.0
other    distant      61.0
Name: patent_citation, dtype: float64

In [37]:
strong_publications.groupby(["source", "depth"])["patent_citation"].mean()


source   depth   
af       adjacent    0.045455
         distant     0.193798
ct_ai    adjacent    0.000000
         distant     0.384615
ct_noai  adjacent    0.571429
         distant     3.068421
other    distant     0.570093
Name: patent_citation, dtype: float64